In [ ]:
! pip install lightfm > /dev/null

In [ ]:
! pip install scikit-optimize

     |████████████████████████████████| 102kB 5.7MB/s 


https://towardsdatascience.com/how-i-would-explain-building-lightfm-hybrid-recommenders-to-a-5-year-old-b6ee18571309

In [ ]:
import pandas as pd
import numpy as np
from skopt import forest_minimize
from lightfm import LightFM, cross_validation
from lightfm import evaluation
from lightfm.evaluation import auc_score
from lightfm.data import Dataset
from scipy import sparse
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Bayer/data_usecase.xlsx')
data.head()

,HCP_id,news_id,Message_id,Message_type,Message_creation_date,Message_TA,news_date,office_or_hospital_based,gender,is_cardiologist,is_gp,years_since_graduation,Message_read,Message_click
0,3569,22,71,Biotech,2019-10-12,Cardio Vascular,2020-08-09,Hospital,False,True,False,10.0,1,0
1,3569,22,70,Biotech,2018-07-31,Global,2020-08-09,Hospital,False,True,False,10.0,0,0
2,5941,22,70,Biotech,2018-07-31,Global,2020-08-09,Hospital,True,True,False,10.0,1,1
3,5941,22,71,Biotech,2019-10-12,Cardio Vascular,2020-08-09,Hospital,True,True,False,10.0,1,1
4,8262,5,19,Biotech,2018-10-12,Cardio Vascular,2020-07-22,Office,True,True,False,10.0,0,0


In [ ]:
def weighted_rating(x):
    a = x['Message_read']
    b = x['Message_click']
    return ((a+b) / 2)

In [ ]:
# Feature Engineering
data['rating'] = data.apply(weighted_rating, axis=1)

# User features

In [ ]:
users = data.loc[:,['HCP_id','office_or_hospital_based','gender','is_cardiologist','is_gp','years_since_graduation']].drop_duplicates(keep='first')

In [ ]:
# Creo una lista unica de combinaciones de features
uf = []
col = ['office_or_hospital_based']*len(users.office_or_hospital_based.unique()) + ['gender']*len(users.gender.unique()) + ['is_cardiologist']*len(users.is_cardiologist.unique()) + ['is_gp']*len(users.is_gp.unique()) + ['years_since_graduation']*len(users.years_since_graduation.unique())
unique_f = list(users.office_or_hospital_based.unique()) + list(users.gender.unique()) + list(users.is_cardiologist.unique()) + list(users.is_gp.unique()) + list(users.years_since_graduation.unique())

for x,y in zip(col, unique_f):
    res = str(x)+ ":" +str(y)
    uf.append(res)
    #print(res)

# Item features

In [ ]:
messages = data.loc[:,['Message_id','Message_type','Message_TA']].drop_duplicates(keep='first')
messages['value'] = 1
messages_wide_type = messages.pivot(index = 'Message_id', columns = 'Message_type', values = 'value', ).fillna(0).add_prefix('type_')
messages_wide_ta = messages.pivot(index = 'Message_id', columns = 'Message_TA', values = 'value').fillna(0).add_prefix('ta_')

messages = messages_wide_type.merge(messages_wide_ta, on = 'Message_id')

messages.columns = messages.columns.str.strip().str.lower().str.replace(' ', '').str.replace('\\/', '')

messages = messages.reset_index()

In [ ]:
# Creo una lista unica de combinaciones de features
itf = []
col = ['type_biotech']*len(messages.type_biotech.unique()) + ['type_clinicaltrialupdate']*len(messages.type_clinicaltrialupdate.unique()) + ['type_medicalstudy']*len(messages.type_medicalstudy.unique()) + ['type_productlaunch']*len(messages.type_productlaunch.unique()) + ['type_serviceapplications']*len(messages.type_serviceapplications.unique()) + ['type_webinar']*len(messages.type_webinar.unique()) + ['ta_cardiovascular']*len(messages.ta_cardiovascular.unique()) + ['ta_else']*len(messages.ta_else.unique()) + ['ta_global']*len(messages.ta_global.unique()) + ['ta_oncology']*len(messages.ta_oncology.unique()) + ['ta_ophtalmology']*len(messages.ta_ophtalmology.unique()) 
unique_f = list(messages.type_biotech.unique()) + list(messages.type_clinicaltrialupdate.unique()) + list(messages.type_medicalstudy.unique()) + list(messages.type_productlaunch.unique()) + list(messages.type_serviceapplications.unique()) + list(messages.type_webinar.unique()) + list(messages.ta_cardiovascular.unique()) + list(messages.ta_else.unique()) + list(messages.ta_global.unique()) + list(messages.ta_oncology.unique()) + list(messages.ta_ophtalmology.unique())

for x,y in zip(col, unique_f):
    res = str(x)+ ":" +str(y)
    itf.append(res)
    #print(res)

# Fitting the dataset

In [ ]:
dataset1 = Dataset()
dataset1.fit(data['HCP_id'].unique(), # all the users
             data['Message_id'].unique(), # all the items
             user_features  = uf,
             item_features = itf
)
dataset1.interactions_shape()

(8349, 100)

In [ ]:
# Using the calculated rating for the weights matrix
(interactions, weights) = dataset1.build_interactions(data[['HCP_id', 'Message_id', 'rating']].itertuples(index=False))
interactions, weights

#The interactions matrix tell us if at all the user interacted with an item and the weights matrix quantifies that particular interaction.

(<8349x100 sparse matrix of type '<class 'numpy.int32'>'
 	with 55543 stored elements in COOrdinate format>,
 <8349x100 sparse matrix of type '<class 'numpy.float32'>'
 	with 55543 stored elements in COOrdinate format>)

In [ ]:
interactions.todense()

matrix([[1, 1, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
weights.todense()

matrix([[0.5, 0. , 0. , ..., 0. , 0. , 0. ],
        [1. , 1. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

## Building users features

In [ ]:
def feature_colon_value_users(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    ll = ['office_or_hospital_based:', 'gender:', 'is_cardiologist:', 'is_gp:', 'years_since_graduation:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [ ]:
ad_subset = users[['office_or_hospital_based', 'gender', 'is_cardiologist', 'is_gp', 'years_since_graduation']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list_users = []
for item in ad_list:
    feature_list_users.append(feature_colon_value_users(item))
    #print(feature_colon_value(item))
#print(f'Final output: {feature_list_users}')

In [ ]:
user_tuple = list(zip(users.HCP_id, feature_list_users))
user_features = dataset1.build_user_features(user_tuple, normalize= False)

In [ ]:
user_features.todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Builing item features

In [ ]:
def feature_colon_value_items(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    ll = ['type_biotech:', 'type_clinicaltrialupdate:', 'type_medicalstudy:', 'type_productlaunch:', 'type_serviceapplications:','type_webinar:','ta_cardiovascular:','ta_else:','ta_global:','ta_oncology:','ta_ophtalmology:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [ ]:
ad_subset_items = messages[['type_biotech', 'type_clinicaltrialupdate', 'type_medicalstudy', 'type_productlaunch', 'type_serviceapplications','type_webinar','ta_cardiovascular','ta_else','ta_global','ta_oncology','ta_ophtalmology']] 
ad_list_items = [list(x) for x in ad_subset_items.values]

feature_list_items = []
for item in ad_list_items:
    feature_list_items.append(feature_colon_value_items(item))
    #print(feature_colon_value(item))
print(f'Final output: {feature_list_items}')

Final output: [['type_biotech:0.0', 'type_clinicaltrialupdate:0.0', 'type_medicalstudy:0.0', 'type_productlaunch:0.0', 'type_serviceapplications:1.0', 'type_webinar:0.0', 'ta_cardiovascular:0.0', 'ta_else:1.0', 'ta_global:0.0', 'ta_oncology:0.0', 'ta_ophtalmology:0.0'], ['type_biotech:0.0', 'type_clinicaltrialupdate:0.0', 'type_medicalstudy:0.0', 'type_productlaunch:1.0', 'type_serviceapplications:0.0', 'type_webinar:0.0', 'ta_cardiovascular:0.0', 'ta_else:0.0', 'ta_global:0.0', 'ta_oncology:0.0', 'ta_ophtalmology:1.0'], ['type_biotech:0.0', 'type_clinicaltrialupdate:1.0', 'type_medicalstudy:0.0', 'type_productlaunch:0.0', 'type_serviceapplications:0.0', 'type_webinar:0.0', 'ta_cardiovascular:0.0', 'ta_else:1.0', 'ta_global:0.0', 'ta_oncology:0.0', 'ta_ophtalmology:0.0'], ['type_biotech:1.0', 'type_clinicaltrialupdate:0.0', 'type_medicalstudy:0.0', 'type_productlaunch:0.0', 'type_serviceapplications:0.0', 'type_webinar:0.0', 'ta_cardiovascular:0.0', 'ta_else:0.0', 'ta_global:1.0', 'ta_

In [ ]:
item_tuple = list(zip(data.Message_id, feature_list_items))
item_features = dataset1.build_item_features(item_tuple, normalize= False)

In [ ]:
item_features.todense()

matrix([[1., 0., 0., ..., 0., 2., 0.],
        [0., 1., 0., ..., 0., 1., 1.],
        [0., 0., 1., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Mapping

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()

# Train Test Split

In [ ]:
train, test = cross_validation.random_train_test_split(interactions, test_percentage=0.2)

In [ ]:
model = LightFM(loss='warp')

model.fit(train,
      user_features = user_features,
      item_features = item_features,
     # sample_weight= weights,
      epochs=10)

In [ ]:
train_precision = evaluation.precision_at_k(model, train, user_features = user_features, item_features = item_features, k=10, num_threads=2).mean()
test_precision  = evaluation.precision_at_k(model, test,  user_features = user_features, item_features = item_features, k=10, num_threads=2).mean()
print('Train@k Precision: {}'.format(train_precision))
print('Test@k Precision: {}'.format(test_precision))

Train@k Precision: 0.44437921047210693
Test@k Precision: 0.11487603932619095


# Hiperoptimization

https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/

In [ ]:
def objective_wsideinfo(params):
    # unpack
    epochs, learning_rate,\
    no_components, item_alpha,\
    scale = params
    
    user_alpha = item_alpha * scale
    model = LightFM(loss='warp',
                    random_state=2020,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train, 
              epochs=epochs,
              user_features=user_features, 
              item_features=item_features,
              num_threads=4, 
              verbose=True)
    
    patks = evaluation.precision_at_k(model,
                                      test,
                                      user_features=user_features, 
                                      item_features=item_features,
                                      train_interactions=None,
                                      k=5, 
                                      num_threads=3)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Weird shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [ ]:
space = [(1, 100), # epochs
         (10**-3, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-5, 10**-3, 'log-uniform'), # item_alpha
         (0.001, 1., 'log-uniform') # user_scaling
        ]

res_fm_itemfeat = forest_minimize(objective_wsideinfo, 
                                  space, 
                                  n_calls=20,
                                  random_state=0,
                                  verbose=True)

Epoch:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration No: 1 started. Evaluating function at random point.


Epoch:   0%|          | 0/84 [00:00<?, ?it/s]

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 26.0750
Function value obtained: -0.0873
Current minimum: -0.0873
Iteration No: 2 started. Evaluating function at random point.


Epoch:   0%|          | 0/13 [00:00<?, ?it/s]

...

Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 1.6888
Function value obtained: -0.0223
Current minimum: -0.1513
Iteration No: 19 started. Searching for the next optimal point.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 1.1952
Function value obtained: -0.1321
Current minimum: -0.1513
Iteration No: 20 started. Searching for the next optimal point.


Epoch: 100%|██████████| 4/4 [00:01<00:00,  2.74it/s]


Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 1.9209
Function value obtained: -0.1343
Current minimum: -0.1513


In [ ]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm_itemfeat.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'item_alpha', 'scaling']
for (p, x_) in zip(params, res_fm_itemfeat.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.15130
Optimal parameters:
epochs: 3
learning_rate: 0.06530550430270628
no_components: 47
item_alpha: 1.9366747805310405e-05
scaling: 0.42496904680239794


# Training on full data

In [ ]:
model_optimized = LightFM(loss='warp',
                          random_state=2020,
                          learning_rate=0.065,
                          no_components=47,
                          user_alpha=0.424,
                          item_alpha=0.00001)

model_optimized.fit(interactions,
                    user_features=user_features,
                    item_features=item_features,
                    sample_weight=weights,
                    num_threads=4, 
                    epochs=3,
                    verbose=True)

Epoch: 100%|██████████| 169/169 [03:51<00:00,  1.37s/it]


# Predictions

In [ ]:
#List of users to predict
users_to_predict = pd.DataFrame(data.HCP_id.drop_duplicates(keep='first'))

## Any message can be predicted

In [ ]:
n_users, n_items = interactions.shape
cant_items_predict = 5
list_items = np.arange(n_items)

li = []
users = []
total = len(users_to_predict)
for row in tqdm(users_to_predict.itertuples(index=False), total=total, position=0, leave=True):
    # Obtengo el mapeo del usuario
    user_id = row.HCP_id
    user_x = user_id_map[user_id]
    # Prescripcion con anteriores
    scores = model_optimized2.predict(user_x, list_items) # means predict for all 
    top_items = list_items[np.argsort(-scores)][:cant_items_predict]
    # Creo las listas de usuarios y recomendaciones
    li.extend(list(top_items))
    users.extend([user_id]*cant_items_predict)

100%|██████████| 8349/8349 [00:01<00:00, 4204.28it/s]


In [ ]:
recomendacion = pd.DataFrame(list(zip(li,users)), 
               columns =['Message_id', 'HCP_id'])
recomendacion

,Message_id,HCP_id
0,31,3569
1,71,3569
2,72,3569
3,99,3569
4,96,3569
...,...,...
41740,62,7477
41741,80,7477
41742,71,7477
41743,99,7477


In [ ]:
recomendacion.to_csv('/content/drive/MyDrive/Bayer/recomendacion_repetidos.csv', index = False)

## Without sending messages that had been sent before

In [ ]:
n_users, n_items = interactions.shape
cant_items_predict = 5
list_items = np.arange(n_items)

li = []
users = []
total = len(users_to_predict)
for row in tqdm(users_to_predict.itertuples(index=False), total=total, position=0, leave=True):
    # Obtengo el mapeo del usuario
    user_id = row.HCP_id
    user_x = user_id_map[user_id]

    # Elimino de la lista de prediccion todos los elementos que ya se enviaron
    items_sent = data.loc[data.HCP_id == user_id,'Message_id'].tolist()
    items_sent_mapped = [item_id_map[i] for i in items_sent]
    list_items_predict = np.array([item for item in data['Message_id'].unique() if item not in items_sent])
    list_items_predict_mapped = list_items[~np.isin(list_items, items_sent_mapped)]

    # Calculo de la prescripcion
    scores = model_optimized.predict(user_x, list_items_predict_mapped) # means predict for all 
    top_items = list_items_predict[np.argsort(-scores)][:cant_items_predict]
  
    # Creo las listas de usuarios y recomendaciones
    li.extend(list(top_items))
    users.extend([user_id]*cant_items_predict)

100%|██████████| 8349/8349 [00:13<00:00, 601.18it/s]


In [ ]:
recomendacion = pd.DataFrame(list(zip(li,users)), 
               columns =['Message_id', 'HCP_id'])
recomendacion

,Message_id,HCP_id
0,49,3569
1,22,3569
2,93,3569
3,15,3569
4,29,3569
...,...,...
41740,49,7477
41741,22,7477
41742,93,7477
41743,15,7477


In [ ]:
recomendacion.to_csv('/content/drive/MyDrive/Bayer/recomendacion_sin_repetidos.csv', index = False)